In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import copy
from WindPy import w
w.start()
from function import *
from nav_research import NavResearch
from pathlib import Path
import os
import shutil
import pyecharts.options as opts
from pyecharts.charts import Line
from pyecharts.globals import ThemeType
import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter('ignore')

plt.rcParams['font.sans-serif'] = ['Microsoft YaHei']
plt.rcParams['axes.unicode_minus'] = False

Welcome to use Wind Quant API for Python (WindPy)!

COPYRIGHT (C) 2024 WIND INFORMATION CO., LTD. ALL RIGHTS RESERVED.
IN NO CIRCUMSTANCE SHALL WIND BE RESPONSIBLE FOR ANY DAMAGES OR LOSSES CAUSED BY USING WIND QUANT API FOR Python.


In [2]:
basic_info = load_data(r"data\产品目录.xlsx")
basic_info

,strategy,fund_code,fund_name,benchmark_code,benchmark_name,threshold
0,主观价值,SQJ763,睿郡财富11号1期,HSI.HI,恒生指数,-0.10
1,主观价值,SS5163,红筹长线3号,000300.SH,沪深300,-0.10
2,CTA,SS0221,均成CTA1号,NH0100.NHF,南华商品指数,-0.10
3,套利,SZH690,辰元优选稳健,H11009.CSI,中证综合债,-0.03


In [3]:
# 删除html文件
def delete_html_files(directory):
    # 遍历目录及其子目录
    for root, dirs, files in os.walk(directory):
        for file in files:
            # 检查文件是否以.html 结尾
            if file.endswith('.html'):
                file_path = os.path.join(root, file)
                try:
                    # 删除文件
                    os.remove(file_path)
                    print(f"Deleted: {file_path}")
                except Exception as e:
                    print(f"Error deleting {file_path}: {e}")

directory_to_clean = r"E:\桌面文件\Vscode\Private_nav_research\docs"
delete_html_files(directory_to_clean)

In [4]:
# 生成新的html文件
for row in basic_info.itertuples(index=False, name=None):
    nav_data_path = os.path.join(r"E:\桌面文件\Vscode\Private_nav_research\data",row[2]+".xlsx")
    demo = NavResearch(nav_data_path,row[0],row[1],row[2],row[3],row[4],row[5])
    demo.get_data()
    demo.get_analysis_table()
    demo.get_html()

均成CTA1号无法推断频率,自动转为周度


In [5]:
# 生成新的index.html文件
def generate_index_html(folder_path: Path):
    # 获取目录及其子目录中的所有 HTML 文件
    html_files = folder_path.rglob("*.html")
    print(html_files)

    # 创建一个新的 index.html 文件
    with open(folder_path.joinpath("index.html"), "w", encoding="utf-8") as f:
        f.write(
            """<html>
<head>
<meta charset="UTF-8">
<title>Value over Time</title>
<style>
    body {
        font-family: Arial, sans-serif;
        font-size: 20px;
        color: blue;
    }
    h2 {
        font-size: 25px;
        color: blue;

    }
    a {
        text-decoration: none;
    }
</style>
</head>
<body>"""
        )
        # 创建一个集合来存储已经处理过的文件夹，避免重复创建链接
        processed_folders = set()
        # 为每个 HTML 文件创建一个链接
        for html_file in html_files:
            if html_file.name == "index.html":
                continue
            relative_path = html_file.relative_to(folder_path)
            # 如果文件夹还没有被处理过，为文件夹创建一个链接
            if html_file.parent not in processed_folders:
                f.write(f"<h2><a>{html_file.parent.name}</a></h2>\n")
                processed_folders.add(html_file.parent)

            # 创建HTML文件的链接
            f.write(
                f'<a href="{relative_path}">{relative_path.name.replace("_nav_analysis","")}</a><br/>\n'
            )

        f.write("</body>\n</html>")

# 使用函数
generate_index_html(Path(r"E:\桌面文件\Vscode\Private_nav_research\docs"))

<generator object Path.rglob at 0x0000019C96DA17A0>


In [6]:
trade_date, weekly_trade_date = generate_trading_date(
            begin_date=np.datetime64("2017-03-01")- np.timedelta64(10, "D"), end_date=np.datetime64("2024-12-06")
        )

In [7]:
nav_df = load_data("data\均成CTA1号.xlsx")

In [8]:
nav_df = nav_df[["日期", "单位净值", "累计净值"]].rename(
    columns={
        "日期": "date",
        "单位净值": "nav_unit",
        "累计净值": "nav_accumulated",
    }
)
nav_df["date"] = pd.to_datetime(nav_df["date"], format="%Y-%m-%d")
nav_df = nav_df.sort_values(by="date")
# start_date, end_date
start_date = pd.Timestamp(nav_df["date"].min()).strftime("%Y-%m-%d")
end_date = pd.Timestamp(nav_df["date"].max()).strftime("%Y-%m-%d")
start_date = start_date
end_date = end_date

In [9]:
nav_df

,date,nav_unit,nav_accumulated
0,2017-03-01,1.0000,1.0000
1,2017-03-02,1.0001,1.0001
2,2017-03-03,1.0001,1.0001
3,2017-03-06,1.0002,1.0002
4,2017-03-07,1.0002,1.0002
...,...,...,...
1498,2024-11-08,6.7900,7.4350
1499,2024-11-15,6.8140,7.4590
1500,2024-11-22,6.7960,7.4410
1501,2024-11-29,6.6780,7.3230


In [10]:
weekly_trade_date

array(['2017-03-03', '2017-03-10', '2017-03-17', '2017-03-24',
       '2017-03-31', '2017-04-07', '2017-04-14', '2017-04-21',
       '2017-04-28', '2017-05-05', '2017-05-12', '2017-05-19',
       '2017-05-26', '2017-06-02', '2017-06-09', '2017-06-16',
       '2017-06-23', '2017-06-30', '2017-07-07', '2017-07-14',
       '2017-07-21', '2017-07-28', '2017-08-04', '2017-08-11',
       '2017-08-18', '2017-08-25', '2017-09-01', '2017-09-08',
       '2017-09-15', '2017-09-22', '2017-09-29', '2017-10-13',
       '2017-10-20', '2017-10-27', '2017-11-03', '2017-11-10',
       '2017-11-17', '2017-11-24', '2017-12-01', '2017-12-08',
       '2017-12-15', '2017-12-22', '2017-12-29', '2018-01-05',
       '2018-01-12', '2018-01-19', '2018-01-26', '2018-02-02',
       '2018-02-09', '2018-02-14', '2018-02-23', '2018-03-02',
       '2018-03-09', '2018-03-16', '2018-03-23', '2018-03-30',
       '2018-04-04', '2018-04-13', '2018-04-20', '2018-04-27',
       '2018-05-04', '2018-05-11', '2018-05-18', '2018-

In [11]:
first_row = nav_df.iloc[[0]].copy()
weekly_trade_date = weekly_trade_date[weekly_trade_date >= nav_df["date"].min()]
nav_data = nav_df.set_index("date")
nav_data = nav_data.reindex(weekly_trade_date, method="ffill")
nav_data = nav_data.reset_index(drop=False)
combined = pd.concat([nav_data, first_row], ignore_index=True)
combined = combined.sort_values(by="date")
combined_unique = combined.drop_duplicates()
combined_unique = combined_unique.reset_index(drop=True)

In [12]:

nav_data = nav_df.set_index("date")
nav_data = nav_data.reindex(trade_date, method="ffill")
nav_data.reset_index(drop=False)

,date,nav_unit,nav_accumulated
0,2017-02-20,NaN,NaN
1,2017-02-21,NaN,NaN
2,2017-02-22,NaN,NaN
3,2017-02-23,NaN,NaN
4,2017-02-24,NaN,NaN
...,...,...,...
1892,2024-12-02,6.678,7.323
1893,2024-12-03,6.678,7.323
1894,2024-12-04,6.678,7.323
1895,2024-12-05,6.678,7.323


In [13]:
nav_df["nav_unit"] = nav_df["nav_unit"] / nav_df["nav_unit"][0]
nav_df["nav_accumulated"] = (
    nav_df["nav_accumulated"] / nav_df["nav_accumulated"][0]
)
nav_df = get_nav_adjusted(nav_df)
